In [1]:
import bittensor as bt
import random
import os
import sys
import numpy as np
import pandas as pd
import hashlib
from collections import defaultdict
from Crypto import Random
from Crypto.Random import random
from Crypto.PublicKey import ECC
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from pprint import pprint, pformat
import binascii


In [2]:
class ECCommitment:
    def __init__(self, g, h):
        self.g = g  # Base point of the curve
        self.h = h  # Another random point on the curve

    def commit(self, m):  # AKA Seal.
        m_val = hash_data(m)  # Compute hash of the data
        r = random.randint(1, 2**256)
        c1 = self.g.__mul__(m_val)
        c2 = self.h.__mul__(r)
        c = c1.__add__(c2)
        print(
            f"Committing: Data = {m}\nHashed Value = {m_val}\nRandom Value = {r}\nComputed Commitment = {c}\n"
        )
        return c, m_val, r

    def open(self, c, m_val, r):
        c1 = self.g.__mul__(m_val)
        c2 = self.h.__mul__(r)
        computed_c = c1.__add__(c2)
        print(
            f"\nOpening: Hashed Value = {m_val}\nRandom Value = {r}\nRecomputed Commitment = {computed_c}\nOriginal Commitment = {c}"
        )
        return computed_c == c


def setup_CRS():
    curve = ECC.generate(curve="P-256")
    g = curve.pointQ  # Base point
    h = ECC.generate(curve="P-256").pointQ  # Another random point
    return g, h


In [3]:
g, h = setup_CRS()
print("g:", g)
print("h:", h)

g: <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ae0f0a0>
h: <Crypto.PublicKey.ECC.EccPoint object at 0x7f4a8b9c3c70>


In [4]:
def make_random_file(name=None, maxsize=1024):
    size = random.randint(32, maxsize)
    data = os.urandom(size)
    if isinstance(name, str):
        with open(name, "wb") as fout:
            fout.write(data)
    else:
        return data

f = random_file = make_random_file()
len(f)

512

In [5]:
def encrypt_data(filename, key):
    """
    Encrypt the data in the given filename using AES-GCM.

    Parameters:
    - filename: str or bytes. If str, it's considered as a file name. If bytes, as the data itself.
    - key: bytes. 16-byte (128-bit), 24-byte (192-bit), or 32-byte (256-bit) secret key.

    Returns:
    - cipher_text: bytes. The encrypted data.
    - nonce: bytes. The nonce used for the GCM mode.
    - tag: bytes. The tag for authentication.
    """

    # If filename is a string, treat it as a file name and read the data
    if isinstance(filename, str):
        with open(filename, "rb") as file:
            data = file.read()
    else:
        data = filename

    # Initialize AES-GCM cipher
    cipher = AES.new(key, AES.MODE_GCM)
    
    # Encrypt the data
    cipher_text, tag = cipher.encrypt_and_digest(data)

    return cipher_text, cipher.nonce, tag

# Example usage
key = get_random_bytes(32)  # 256-bit key
encrypted_data, nonce, tag = encrypt_data(random_file, key)
encrypted_data

b'\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7\'\xd1\xa9t0(\'\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99e\x94qJ\x83H\xec\x85\xbc\xda\x8e\x82\x14\xb8v7`\xe8\xc8/(\xd2B)\xa9M\xb1s\x7f\xc9\xda\x94\xa2\xb8\x84\xc9\x9a\n\xdb\xe3\xff\xdf\xfaw\xf3~\x1eJ\xb3J\xe2\xfa\xfc\xf9\\`\xa0\xbd\xdaas\xf3A\x94z\x8e\xa7\x15\xa3\x1bv"\x00\r\x01\xa7\xda\xe1t\x10\xa3\x90F\xbe\xbc?73\x01\xbd\xce\xcf\x9cM\xc7\xd2\xa8\xdb\xfd\x96\x9c\xd5\xc1<\x16\xa4h{o\xe0\x9f\xf1\x9dk\x9fJ\xa7\xdc\xf6\x81\x92\xb5E\x127~\x16%q\xaaU\xbc-\x9d\xf1\xa0\x9d\x8a\x01\nm\x9cU5pE\xe6\xae\x8c<\r\xf8\x9a\xa4T\xa5\xc2\xc7\xe5\xf1\xb8\x94\xe2g\x19\x8f\x9c;\x9b\x7fFO\xee\xf6wB\xe9\xaf;-\x86\x9c\x81\xaa\xac\x91S\xc0\x08(\x88\xf8\x1eJ\xc6\x9a\xf2s\x85Hj9\x98\\G"\x1b\x90vn&\x0b\x1f\x00\x16\x87\xd9?\xe3\xe1\xc3\x96\xeb\xd4\xc6Q

In [6]:
# Hash encrypted data E with SHA3-256 (args: encrypted data E) to get hash H
def hash_data(data):
    if not isinstance(data, (bytes, bytearray)):
        data_str = str(data)
        data = data_str.encode()
    h = hashlib.sha3_256(data).hexdigest()
    return int(h, 16)

H = hash_data(encrypted_data)
H

12117966378284384701652859315035581497141308452771122956723054776000079563328

In [7]:
# Determine a random chunksize between 2kb-128kb (random sample from this range) store as chunksize_E
def get_random_chunksize(maxsize=128):
    return random.randint(2, maxsize)

bytesize = sys.getsizeof(encrypted_data)
print("bytesize:", bytesize)
chunksize = get_random_chunksize(bytesize // 4)
print("chunksize:", chunksize)

bytesize: 545
chunksize: 107


In [8]:
def chunk_data(data, chunksize: int):
    for i in range(0, len(data), chunksize):
        yield data[i : i + chunksize]

chunks = list(chunk_data(encrypted_data, chunksize))
print("n chunks:", len(chunks))
pprint(f"chunks: {chunks}")

n chunks: 5
('chunks: '
 '[b"\\xa9]\\xc8\\x1d\\xf1\\xa0\\xac\\xf8&\\x93\\xb4\\xd0\\xa8\\x1d_\\x1d(X\\xc7\'\\xd1\\xa9t0(\'\\xb6\\x17\\x0e\\x17\\x99bPf\\xe0\\xa0\\xa6\\xc3\\xb9\\xcc\\xe1\\x97,n\\xfd\\xc3.\\x05\\x08\\x14)\\x15\\x8d\\xcf\\x8c8\\xf5\\xee\\x1fRn\\xfa\\xb9\\x96\\xe6\\xb5\\xcf\\x90\\xae\\xa8\\xa7U:``CT[\\x1e\\xcc\\xc2\\xed\\x10\\xe9RR\\xcb>\\xdcsbJ\\x18\\x9f\\x17\\xf0\\xb2\\xe9\\x9b]\\x9c\\xe0\\xd7\\xcbR7\\x99", '
 'b\'e\\x94qJ\\x83H\\xec\\x85\\xbc\\xda\\x8e\\x82\\x14\\xb8v7`\\xe8\\xc8/(\\xd2B)\\xa9M\\xb1s\\x7f\\xc9\\xda\\x94\\xa2\\xb8\\x84\\xc9\\x9a\\n\\xdb\\xe3\\xff\\xdf\\xfaw\\xf3~\\x1eJ\\xb3J\\xe2\\xfa\\xfc\\xf9\\\\`\\xa0\\xbd\\xdaas\\xf3A\\x94z\\x8e\\xa7\\x15\\xa3\\x1bv"\\x00\\r\\x01\\xa7\\xda\\xe1t\\x10\\xa3\\x90F\\xbe\\xbc?73\\x01\\xbd\\xce\\xcf\\x9cM\\xc7\\xd2\\xa8\\xdb\\xfd\\x96\\x9c\\xd5\\xc1<\\x16\\xa4h\', '
 'b\'{o\\xe0\\x9f\\xf1\\x9dk\\x9fJ\\xa7\\xdc\\xf6\\x81\\x92\\xb5E\\x127~\\x16%q\\xaaU\\xbc-\\x9d\\xf1\\xa0\\x9d\\x8a\\x01\\nm\\x9cU5pE\\xe6\\xae\\x8c<\\r\\xf8\\

In [9]:
committer = ECCommitment(g, h)
committer

In [10]:
class MerkleTree(object):
    def __init__(self, hash_type="sha3_256"):
        hash_type = hash_type.lower()
        if hash_type in [
            "sha256",
            "sha224",
            "sha384",
            "sha512",
            "sha3_256",
            "sha3_224",
            "sha3_384",
            "sha3_512",
        ]:
            self.hash_function = getattr(hashlib, hash_type)
        else:
            raise Exception("`hash_type` {} nor supported".format(hash_type))

        self.reset_tree()

    def _to_hex(self, x):
        try:  # python3
            return x.hex()
        except:  # python2
            return binascii.hexlify(x)

    def reset_tree(self):
        self.leaves = list()
        self.levels = None
        self.is_ready = False

    def add_leaf(self, values, do_hash=False):
        self.is_ready = False
        # check if single leaf
        if not isinstance(values, tuple) and not isinstance(values, list):
            values = [values]
        for v in values:
            if do_hash:
                v = v.encode("utf-8")
                v = self.hash_function(v).hexdigest()
            v = bytearray.fromhex(v)
            self.leaves.append(v)

    def get_leaf(self, index):
        return self._to_hex(self.leaves[index])

    def get_leaf_count(self):
        return len(self.leaves)

    def get_tree_ready_state(self):
        return self.is_ready

    def _calculate_next_level(self):
        solo_leave = None
        N = len(self.levels[0])  # number of leaves on the level
        if N % 2 == 1:  # if odd number of leaves on the level
            solo_leave = self.levels[0][-1]
            N -= 1

        new_level = []
        for l, r in zip(self.levels[0][0:N:2], self.levels[0][1:N:2]):
            new_level.append(self.hash_function(l + r).digest())
        if solo_leave is not None:
            new_level.append(solo_leave)
        self.levels = [
            new_level,
        ] + self.levels  # prepend new level

    def make_tree(self):
        self.is_ready = False
        if self.get_leaf_count() > 0:
            self.levels = [
                self.leaves,
            ]
            while len(self.levels[0]) > 1:
                self._calculate_next_level()
        self.is_ready = True

    def get_merkle_root(self):
        if self.is_ready:
            if self.levels is not None:
                return self._to_hex(self.levels[0][0])
            else:
                return None
        else:
            return None

    def get_proof(self, index):
        if self.levels is None:
            return None
        elif not self.is_ready or index > len(self.leaves) - 1 or index < 0:
            return None
        else:
            proof = []
            for x in range(len(self.levels) - 1, 0, -1):
                level_len = len(self.levels[x])
                if (index == level_len - 1) and (
                    level_len % 2 == 1
                ):  # skip if this is an odd end node
                    index = int(index / 2.0)
                    continue
                is_right_node = index % 2
                sibling_index = index - 1 if is_right_node else index + 1
                sibling_pos = "left" if is_right_node else "right"
                sibling_value = self._to_hex(self.levels[x][sibling_index])
                proof.append({sibling_pos: sibling_value})
                index = int(index / 2.0)
            return proof


    def update_leaf(self, index, new_value):
        """Update a specific leaf in the tree and propagate changes upwards."""
        if not self.is_ready:
            return None
        new_value = bytearray.fromhex(new_value)
        self.levels[-1][index] = new_value
        for x in range(len(self.levels) - 1, 0, -1):
            parent_index = index // 2
            left_child = self.levels[x][parent_index * 2]
            try:
                right_child = self.levels[x][parent_index * 2 + 1]
            except IndexError:
                right_child = bytearray()
            self.levels[x-1][parent_index] = self.hash_function(left_child + right_child).digest()
            index = parent_index

def validate_merkle_proof(proof, target_hash, merkle_root):
    merkle_root = bytearray.fromhex(merkle_root)
    target_hash = bytearray.fromhex(target_hash)
    if len(proof) == 0:
        return target_hash == merkle_root
    else:
        proof_hash = target_hash
        for p in proof:
            try:
                # the sibling is a left node
                sibling = bytearray.fromhex(p["left"])
                proof_hash = hashlib.sha3_256(sibling + proof_hash).digest()
            except:
                # the sibling is a right node
                sibling = bytearray.fromhex(p["right"])
                proof_hash = hashlib.sha3_256(proof_hash + sibling).digest()
        return proof_hash == merkle_root


In [11]:
def commit_data(committer, data_chunks):
    merkle_tree = MerkleTree()
    commitments = defaultdict(lambda: [None] * len(data_chunks))
    # commitments = {}

    for index, chunk in enumerate(data_chunks):
        c, m_val, r = committer.commit(chunk)
        commitments[index] = {
            "index": index,
            "hash": m_val,
            "data_chunk": chunk,
            "point": c,
            "randomness": r,
            "merkle_proof": None,
        }
        merkle_tree.add_leaf(ecc_point_to_hex(c))

    merkle_tree.make_tree()
    return {merkle_tree.get_merkle_root(): {'commitments': commitments, 'merkle_tree': merkle_tree}}

def ecc_point_to_hex(point):
    point_str = "{},{}".format(point.x, point.y)
    return binascii.hexlify(point_str.encode()).decode()

def hex_to_ecc_point(hex_str, curve):
    point_str = binascii.unhexlify(hex_str).decode()
    x, y = map(int, point_str.split(","))
    return ECC.EccPoint(curve, x, y)

store_data = commit_data
commitments_E = store_data(committer, chunks)
commitments_E
# TODO: once this is generaetd, we send it back to the Validator minus the random values
# {merkle_root: (data_chunks, hashes, commitment_points)} 

Committing: Data = b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99"
Hashed Value = 112338643405848501298939711485700548512527027948856895189884594257455301547077
Random Value = 48440412178191298709631090262485133591133900680607127431851860250135419929156
Computed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab02a70>

Committing: Data = b'e\x94qJ\x83H\xec\x85\xbc\xda\x8e\x82\x14\xb8v7`\xe8\xc8/(\xd2B)\xa9M\xb1s\x7f\xc9\xda\x94\xa2\xb8\x84\xc9\x9a\n\xdb\xe3\xff\xdf\xfaw\xf3~\x1eJ\xb3J\xe2\xfa\xfc\xf9\\`\xa0\xbd\xdaas\xf3A\x94z\x8e\xa7\x15\xa3\x1bv"\x00\r\x01\xa7\xda\xe1t\x10\xa3\x90F\xbe\xbc?73\x01\xbd\xce\xcf\x9cM\xc7\xd2\xa8\xdb\xfd\x96\x9c\xd5\xc1<\x16\xa4h'
Hashed Value = 10920654002659217021292821

{'f2804064f642bd401c04b0c629d4ccc311a93f77c66fb058aca333298c25c969': {'commitments': defaultdict(<function __main__.commit_data.<locals>.<lambda>()>,
              {0: {'index': 0,
                'hash': 112338643405848501298939711485700548512527027948856895189884594257455301547077,
                'data_chunk': b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99",
                'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f491ab02a70>,
                'randomness': 48440412178191298709631090262485133591133900680607127431851860250135419929156,
                'merkle_proof': None},
               1: {'index': 1,
                'hash': 109206540026592170212928219278061917887631092984523302797960050338917567181456,
 

In [12]:
keys = list(commitments_E.keys())
keys

['f2804064f642bd401c04b0c629d4ccc311a93f77c66fb058aca333298c25c969']

In [13]:
inner_dict = commitments_E[keys[0]]
inner_dict

{'commitments': defaultdict(<function __main__.commit_data.<locals>.<lambda>()>,
             {0: {'index': 0,
               'hash': 112338643405848501298939711485700548512527027948856895189884594257455301547077,
               'data_chunk': b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99",
               'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f491ab02a70>,
               'randomness': 48440412178191298709631090262485133591133900680607127431851860250135419929156,
               'merkle_proof': None},
              1: {'index': 1,
               'hash': 109206540026592170212928219278061917887631092984523302797960050338917567181456,
               'data_chunk': b'e\x94qJ\x83H\xec\x85\xbc\xda\x8e\x82\x14\xb8v7`

In [14]:
hashes = inner_dict["commitments"]["hashes"]
assert len(hashes) == len(chunks)

In [15]:
def get_challenge_indices(num_chunks, factor=0.1):
    return random.sample(list(range(num_chunks)), int(1 + num_chunks * factor))

ci = challenge_indices = get_challenge_indices(len(chunks))
ci

[0]

In [16]:
def get_merkle_root_to_challenge(commitments):
    merkle_roots = list(commitments.keys())
    return random.choice(merkle_roots)

mrc = merkle_root_challenge = get_merkle_root_to_challenge(commitments_E)
mrc

'f2804064f642bd401c04b0c629d4ccc311a93f77c66fb058aca333298c25c969'

In [17]:
responses = []
challenge_data = commitments_E[merkle_root_challenge]
merkle_tree = challenge_data['merkle_tree']
for i in challenge_indices:
    challenge_data['commitments'][i]['merkle_proof'] = merkle_tree.get_proof(i)
    responses.append(challenge_data['commitments'][i])

challenge_response = {merkle_root_challenge: responses}
challenge_response

{'f2804064f642bd401c04b0c629d4ccc311a93f77c66fb058aca333298c25c969': [{'index': 0,
   'hash': 112338643405848501298939711485700548512527027948856895189884594257455301547077,
   'data_chunk': b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99",
   'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f491ab02a70>,
   'randomness': 48440412178191298709631090262485133591133900680607127431851860250135419929156,
   'merkle_proof': [{'right': '36333833383435373835303233383339333038313139323933303836383134353134383634303530303437333535353737383532393333353736343136303631383239393033303531393439372c313039363539363536383030363833373332373136353539353335373134343036353132383338383230343838353332303235353536373137303239383232373839343831

In [18]:
merkle_root = list(challenge_response.keys())[0]
commitments = challenge_response[merkle_root]
print("merkle_root:", merkle_root)
commitments

merkle_root: f2804064f642bd401c04b0c629d4ccc311a93f77c66fb058aca333298c25c969


[{'index': 0,
  'hash': 112338643405848501298939711485700548512527027948856895189884594257455301547077,
  'data_chunk': b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99",
  'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f491ab02a70>,
  'randomness': 48440412178191298709631090262485133591133900680607127431851860250135419929156,
  'merkle_proof': [{'right': '36333833383435373835303233383339333038313139323933303836383134353134383634303530303437333535353737383532393333353736343136303631383239393033303531393439372c313039363539363536383030363833373332373136353539353335373134343036353132383338383230343838353332303235353536373137303239383232373839343831353936333539393439'},
   {'right': '4c7a97301f0225a413c8dd96d791943020b546

In [19]:
len(commitments)

1

In [20]:
commitment_i = commitments[0]
commitment_i

{'index': 0,
 'hash': 112338643405848501298939711485700548512527027948856895189884594257455301547077,
 'data_chunk': b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99",
 'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f491ab02a70>,
 'randomness': 48440412178191298709631090262485133591133900680607127431851860250135419929156,
 'merkle_proof': [{'right': '36333833383435373835303233383339333038313139323933303836383134353134383634303530303437333535353737383532393333353736343136303631383239393033303531393439372c313039363539363536383030363833373332373136353539353335373134343036353132383338383230343838353332303235353536373137303239383232373839343831353936333539393439'},
  {'right': '4c7a97301f0225a413c8dd96d791943020b546e357a2a

In [21]:
merkle_proof_i = commitments[0]['merkle_proof']
merkle_proof_i

[{'right': '36333833383435373835303233383339333038313139323933303836383134353134383634303530303437333535353737383532393333353736343136303631383239393033303531393439372c313039363539363536383030363833373332373136353539353335373134343036353132383338383230343838353332303235353536373137303239383232373839343831353936333539393439'},
 {'right': '4c7a97301f0225a413c8dd96d791943020b546e357a2aa472a0208e1a1fcebf9'},
 {'right': '35393937343439313033363833373031303538373135313333373631333032323736373931343939343436333337303737373639343035343332373038373133313636383435303334363535352c3530393131383138383334323033323632303031313039353932383833303239363238323430323638333336363135323031303632373438343931313936363333313530383736383937333539'}]

In [22]:
commitments[0]

{'index': 0,
 'hash': 112338643405848501298939711485700548512527027948856895189884594257455301547077,
 'data_chunk': b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99",
 'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f491ab02a70>,
 'randomness': 48440412178191298709631090262485133591133900680607127431851860250135419929156,
 'merkle_proof': [{'right': '36333833383435373835303233383339333038313139323933303836383134353134383634303530303437333535353737383532393333353736343136303631383239393033303531393439372c313039363539363536383030363833373332373136353539353335373134343036353132383338383230343838353332303235353536373137303239383232373839343831353936333539393439'},
  {'right': '4c7a97301f0225a413c8dd96d791943020b546e357a2a

In [23]:
for commitment_i in commitments:
    index = commitment_i['index']
    commitment = commitment_i['point']
    data = commitment_i['data_chunk']
    r = commitment_i['randomness']
    merkle_proof_i = commitment_i['merkle_proof']

    if not committer.open(commitment, hash_data(data), r):
        print(f"Opening commitment {index} failed")

    if not validate_merkle_proof(
        merkle_proof_i, ecc_point_to_hex(commitment_i['point']), merkle_root
    ):
        print(f"Merkle proof {index} validation failed")


Opening: Hashed Value = 112338643405848501298939711485700548512527027948856895189884594257455301547077
Random Value = 48440412178191298709631090262485133591133900680607127431851860250135419929156
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab00100>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab02a70>


In [24]:
def verify(committer, commitments, merkle_root):
    for commitment_i in commitments:
        index = commitment_i['index']
        commitment = commitment_i['point']
        data = commitment_i['data_chunk']
        r = commitment_i['randomness']
        merkle_proof_i = commitment_i['merkle_proof']

        if not committer.open(commitment, hash_data(data), r):
            print(f"Opening commitment {index} failed")
            return False

        if not validate_merkle_proof(
            merkle_proof_i, ecc_point_to_hex(commitment_i['point']), merkle_root
        ):
            print(f"Merkle proof {index} validation failed")
            return False

    return True

In [25]:
verify(committer, commitments, merkle_root)


Opening: Hashed Value = 112338643405848501298939711485700548512527027948856895189884594257455301547077
Random Value = 48440412178191298709631090262485133591133900680607127431851860250135419929156
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab02b60>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab02a70>


True

In [26]:
def recommit_data(committer, challenge_indices, merkle_tree, data):
    # new_commitments = {}
    new_commitments = []
    for i in challenge_indices:
        c, m_val, r = committer.commit(data[i])
        commitment_hash = hash_data(ecc_point_to_hex(c)) # Assuming a hash_function is available.
        new_commitments.append( {
            "index": i,
            "hash": commitment_hash,
            "data_chunk": data[i],
            "point": c,
            "randomness": r,
            "merkle_proof": None
        })
        merkle_tree.update_leaf(i, ecc_point_to_hex(c))
    new_merkle_root = merkle_tree.get_merkle_root()
    return new_merkle_root, new_commitments

new_merkle_root, new_commitments = recommit_data(committer, challenge_indices, merkle_tree, chunks)
print("new_merkle_root:", new_merkle_root)
new_commitments

Committing: Data = b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99"
Hashed Value = 112338643405848501298939711485700548512527027948856895189884594257455301547077
Random Value = 84677239110191786396945898098136819935221217544383439955726374883337643675469
Computed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab03010>

new_merkle_root: 6c855fdaa908787d27ff727966e43daa083c026bef6b50516d4bd7ac45cc90f9


[{'index': 0,
  'hash': 67524870516429278462838599991431937271735840779341060905602231448213289142482,
  'data_chunk': b"\xa9]\xc8\x1d\xf1\xa0\xac\xf8&\x93\xb4\xd0\xa8\x1d_\x1d(X\xc7'\xd1\xa9t0('\xb6\x17\x0e\x17\x99bPf\xe0\xa0\xa6\xc3\xb9\xcc\xe1\x97,n\xfd\xc3.\x05\x08\x14)\x15\x8d\xcf\x8c8\xf5\xee\x1fRn\xfa\xb9\x96\xe6\xb5\xcf\x90\xae\xa8\xa7U:``CT[\x1e\xcc\xc2\xed\x10\xe9RR\xcb>\xdcsbJ\x18\x9f\x17\xf0\xb2\xe9\x9b]\x9c\xe0\xd7\xcbR7\x99",
  'point': <Crypto.PublicKey.ECC.EccPoint at 0x7f491ab03010>,
  'randomness': 84677239110191786396945898098136819935221217544383439955726374883337643675469,
  'merkle_proof': None}]

In [27]:
# Generate merkle proof for updated indices
for nc in new_commitments:
    nc['merkle_proof'] = merkle_tree.get_proof(nc['index'])
nc['merkle_proof']

[{'right': '36333833383435373835303233383339333038313139323933303836383134353134383634303530303437333535353737383532393333353736343136303631383239393033303531393439372c313039363539363536383030363833373332373136353539353335373134343036353132383338383230343838353332303235353536373137303239383232373839343831353936333539393439'},
 {'right': '4c7a97301f0225a413c8dd96d791943020b546e357a2aa472a0208e1a1fcebf9'},
 {'right': '35393937343439313033363833373031303538373135313333373631333032323736373931343939343436333337303737373639343035343332373038373133313636383435303334363535352c3530393131383138383334323033323632303031313039353932383833303239363238323430323638333336363135323031303632373438343931313936363333313530383736383937333539'}]

In [28]:
# Reverify new commitments to make sure we're solid
verify(committer, new_commitments, new_merkle_root)


Opening: Hashed Value = 112338643405848501298939711485700548512527027948856895189884594257455301547077
Random Value = 84677239110191786396945898098136819935221217544383439955726374883337643675469
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab019c0>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7f491ab03010>


True